# Creating a GIS layer for Finnish Health Districts


In this tutorial, we will create boundaries of Finnish health districts (*sairaanhoitopiiri* in Finnish) by dissolving municipality boundaries into larger entities. Main processing steps include a table join and dissolving the municipality geometries into larger entities.

We will combine information from [municipality polygons](https://www.stat.fi/org/avoindata/paikkatietoaineistot/vaesto_tilastointialueittain.html) from Statistics Finland and a [list of health care districts](https://www.kuntaliitto.fi/sosiaali-ja-terveysasiat/sairaanhoitopiirien-jasenkunnat) by the Finnish Municipality authority Kuntaliitto.

Importing required python packages:

In [ ]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt

## Read in data
- **Municipality polygons** from Statistics Finland web feature service: https://www.stat.fi/org/avoindata/paikkatietoaineistot/kuntapohjaiset_tilastointialueet.html
    - wfs: http://geo.stat.fi/geoserver/tilastointialueet/wfs?
    - feature: `tilastointialueet:kunta1000k` (most recent information about municipality polygons)





In [ ]:
# For available features, see http://geo.stat.fi/geoserver/tilastointialueet/wfs?request=GetCapabilities
url = "http://geo.stat.fi/geoserver/tilastointialueet/wfs?request=GetFeature&typename=tilastointialueet:kunta1000k&outputformat=JSON"
geodata = 

In [ ]:
# Check length (there are 310 municipalities in Finland in 2020)


In [ ]:
#Select and rename columns
geodata.rename(columns={'kunta':'code'}, inplace=True)
geodata = geodata[['code','name', 'geometry']]
geodata.head()

In [ ]:
# Plot the data


In [ ]:
# Check data types


- **Finnish municipalities with health district information** as an Excel spreadsheet 
    - Downloaded from: https://www.kuntaliitto.fi/sosiaali-ja-terveysasiat/sairaanhoitopiirien-jasenkunnat in March 2020. 
    - File `Shp_jäsenkunnat_2020.xls`, sheet `kunnat_shp_2020_ aakkosjärj.` This is the original unaltered file.
    - In this file, "shp" stands for "sairaanhoitopiiri" (health district in Finnish)
    
*Note: this data set does not include Åland (Ahvenanmaa). Åland municipalities are added in the later step.*

Excel files often come with additional formatting such as metadata on the first lines of the data array. This is why it is a good idea to download the file on your own computer and have a look at the data structure before reading in the file using Python.
It is also often a good idea to save the file as a csv file before reading in the data. However, it is also possible to read in data directly from Excel. For this, you need to have the xlrd module installed:

```
conda install -c conda-forge xlrd
```

Now we are ready to read in the data using pandas.

In the case of this health districts excel the header is located on the 4th row (index 3) of the excel spreadsheet. 

In [ ]:
# Read in the excel spreadsheet


In addition, the first row after the header is empty. We can get rid of it using the dropna() -function:

Check number of rows (16 Åland municipalities are missing)

The data needs some fixing and cleaning after reading the excel sheet

In [ ]:
# Rename columns from Finnish to English 
data.rename(columns={"kunta-\nkoodi":"code", 'sairaanhoitopiiri':'healthCareDistrict'}, inplace=True)

# Select only useful columns
data = data[['code','healthCareDistrict']]

In [ ]:
data

Looks better! Now we need to prepare the data for table join. We will use the municipality code as the common key.

In [ ]:
# Check data types
data.dtypes

The code column is currently a floating point number. We need to modify these codes so that they match the ones in the spatial data:

In [ ]:
# Example using one code
number = data.at[1, "code"]
number

In [ ]:
# Conver this number to character string 020


Let's apply this process on all rows at once, and take into account different number of digits:

In [ ]:
# Truncate and convert to character string

# Add missing zeros to municipality codes


## Join Health district info to the municipality polygons

In [ ]:
# Merge health district info to geodata using "code" as the common key


Looks good! However, Municipalities in the Åland island did not have a matching health care district in the data. Let's have a closer look: 

In [ ]:
# List all municipalities that lack health district info:


In [ ]:
# Update "Ahvenanmaa" as the health care district for Åland municipalities (16 municipalities in total)


Check the count of municipalities per health care disctrict

## Create polygons for health care districts 

In [ ]:
# Dissolve (=combine) municipality polygon geometries for each health care district


In [ ]:
# Reset index


In [ ]:
# Visualize


In [ ]:
# Write GeoJSON in original projection
fp1 = "healthDistrictsEPSG3067.geojson"


In [ ]:
# Re-project to WGS84 and save again
fp2 = "healthDistrictsEPSG4326.geojson"


That's it! You can elaborate this workflow by joining additional data. For example, if you join population info per municipality you can sum it up for each health care district using the `aggfunc=sum` argument to get population count per health care district.